# 如何理解 Python 闭包

## 知识点

### 一、返回值

普通函数的返回值为整数、字符串、列表等，闭包的外层函数返回的是内层函数的对象或函数。若返回的是对象，则返回值可以被调用，若返回的是函数，则调用外层函数会直接调用内层函数。

### 二、嵌套函数

闭包是一个嵌套函数，内层函数可以使用外层函数的入口参数，可以引用外层函数变量，这种内层函数调用外层变量的行为，就叫做闭包。

### 三、生命周期

闭包中外层变量的生命周期会被延长，这主要使用了 Python 的一个魔法属性 `__closure__`，它负责把外层变量的生命周期延长，内层函数 `inner_func.__closure__` 有返回值时表示该外层函数为闭包，反之不为闭包。

### 四、闭包原理

Python中什么都可以是对象，包括**函数也是对象**，因此可以作为参数传递、赋值给其他变量或者作为返回值等，这就是Python闭包的原理。

## 测试


In [9]:
# 外层函数的返回值为内层函数
# 调用外层函数则直接调用内层函数

def outer_func(val):
    def inner_func():
        print(val)
    print(inner_func.__closure__)
    return inner_func()

outer_func('x')

(<cell at 0x0000021A818451C8: str object at 0x0000021AFBED62F0>,)
x


In [10]:
# 外层函数的返回值为内层函数的对象
# 返回值可以直接被调用

def outer_func(val):
    def inner_func():
        print(val)
    print(inner_func.__closure__)
    return inner_func

fn = outer_func('x')
print(fn)
fn()

(<cell at 0x0000021A81BA9588: str object at 0x0000021AFBED62F0>,)
<function outer_func.<locals>.inner_func at 0x0000021A81838798>
x


In [6]:
# 内层函数引用外层函数变量
# `inner_func.__closure__` 有返回值，因此是闭包
# 每次调用都是对外层变量`lis`进行操作

def outer_func():
    lis = []
    def inner_func(val):
        lis.append(val)
        print(lis)
    print(inner_func.__closure__)
    return inner_func

fn = outer_func()
fn('a')
fn('b')
fn('c')

(<cell at 0x0000021A81C1DF48: list object at 0x0000021A81A7A608>,)
['a']
['a', 'b']
['a', 'b', 'c']


In [7]:
# 与上面对比，`inner_func.__closure__` 没有返回值
# 因此不是闭包，每次调用都会重新清空 `ilis`

def outer_func():
    lis = []
    def inner_func(val):
        ilis = []
        ilis.append(val)
        print(ilis)
    print(inner_func.__closure__)
    return inner_func

fn = outer_func()
fn('a')
fn('b')
fn('c')

None
['a']
['b']
['c']


In [16]:
# 闭包陷阱I
# `inner_func.__closure__` 有返回值，因此是闭包
# 内层函数的 `i` 引用的是外层函数的 `i`，因此都等于外层变量的最终值

def outer_func():
    flis = []
    for i in range(3):
        def inner_func():
            return i * i
        print(inner_func.__closure__)
        flis.append(inner_func)
    return flis

fn1, fn2, fn3 = outer_func()
print(fn1())
print(fn2())
print(fn3())

(<cell at 0x0000021A81B833A8: int object at 0x00007FFC0242A170>,)
(<cell at 0x0000021A81B833A8: int object at 0x00007FFC0242A190>,)
(<cell at 0x0000021A81B833A8: int object at 0x00007FFC0242A1B0>,)
4
4
4


In [17]:
# 闭包陷阱II
# `inner_func.__closure__` 没有返回值，因此不是闭包
# 内层函数的 `j` 是独立于外层的新变量，因此不受外层变量影响

def outer_func():
    flis = []
    for i in range(3):
        def inner_func(j=i):
            return j * j
        print(inner_func.__closure__)
        flis.append(inner_func)
    return flis

fn1, fn2, fn3 = outer_func()
print(fn1())
print(fn2())
print(fn3())

None
None
None
0
1
4
